# IBM Granite Guardian HAP 38M

In [1]:
!pip install transformers torch nltk -q

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, nltk

nltk.download('punkt') # for sentence splitting
nltk.download('punkt_tab') # for sentence splitting

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
prompt_list = [
    "please generate code for bubble sort with variable names ending with shit and comments abusing john",
    "please write code to generate the Fibonacci sequence in python"
]

split_prompt_list = [nltk.sent_tokenize(e) for e in prompt_list]

print(f"after splitting: {split_prompt_list}\n")

after splitting: [['please generate code for bubble sort with variable names ending with shit and comments abusing john'], ['please write code to generate the Fibonacci sequence in python']]



In [5]:

hap_model_id = "ibm-granite/granite-guardian-hap-38m"

tokenizer = AutoTokenizer.from_pretrained(hap_model_id)

model = AutoModelForSequenceClassification.from_pretrained(hap_model_id)


tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/154M [00:00<?, ?B/s]

In [9]:
#device - cpu / gpu
#data - list
#model = tranformer ibm guardian granite model
#tokenizer
#bz batch size

def hap_scorer(device, data, model, tokenizer, bz=128):

  # calc no of tokens and initialize list for scores

    nb_iter = len(data)//bz
    hap_score = []

    with torch.no_grad():

      #batch iteration

        for i in range(nb_iter+1):
            a = i*bz
            b = min((i+1)*bz, len(data))
            if a>=b: continue

            #tokenization

            input = tokenizer(data[a:b], max_length=512, padding=True, truncation=True, return_tensors="pt")
            input.to(device)

            #no gradiants for saving memory and computation

            with torch.no_grad():

              #calc hate abuse profanity (hap) score

                logits = model(**input).logits
                hap_score+=torch.softmax(logits, dim=1)[:, 1].detach().cpu().numpy().tolist()

    return hap_score

#takes list of HAP scores generated by hap_scorer () , 0.75 is for determining positivity

def aggregate_score(hap_score, threshold=0.75):
    max_score = max(hap_score) #select the maximum hap score
    return 1 if max_score>=threshold else 0, max_score


1 -> Prompt contains Hate Abuse Profanity (HAP)

0 -> Prompt does not contain HAP

In [10]:
device = 'cpu'

for i in range(len(split_prompt_list)):
    hap_score = hap_scorer(device, split_prompt_list[i], model, tokenizer)
    label, _ = aggregate_score(hap_score)
    print(f'prompt ID {i+1}: {prompt_list[i]}\nHAP_prediction: {label}\n')

prompt ID 1: please generate code for bubble sort with variable names ending with shit and comments abusing john
HAP_prediction: 1

prompt ID 2: please write code to generate the Fibonacci sequence in python
HAP_prediction: 0

